In [16]:
import numpy as np
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [13]:
import librosa
import numpy as np

SAMPLE_RATE = 16000

def load_and_preprocess(file_path, sr=SAMPLE_RATE):
    y, _ = librosa.load(file_path, sr=sr)
    y, _ = librosa.effects.trim(y, top_db=30)
    y = y / np.max(np.abs(y)) if np.max(np.abs(y)) > 0 else y
    return y

def extract_features(y, sr):
    mfcc     = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    rolloff  = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rms      = librosa.feature.rms(y=y)
    zcr      = librosa.feature.zero_crossing_rate(y=y)

    try:
        f0 = librosa.yin(y, sr=sr, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
        f0 = f0.reshape(1, -1)
    except:
        f0 = np.zeros((1, mfcc.shape[1]))

    T = mfcc.shape[1]
    def resize(f): return f[:, :T] if f.shape[1] >= T else np.pad(f, ((0, 0), (0, T - f.shape[1])))

    all_features = np.vstack([
        mfcc,
        resize(rolloff),
        resize(rms),
        resize(zcr),
        resize(f0)
    ])
    return all_features.T  # shape: (T, feature_dim)


In [14]:
# Store features separately
speech_sequences, speech_labels = [], []
song_sequences, song_labels = [], []

ravdess_emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

def process_ravdess(path, tag):
    sequences, labels = [], []
    count = 0
    for actor_folder in os.listdir(path):
        folder_path = os.path.join(path, actor_folder)
        if not os.path.isdir(folder_path):
            continue
        for file in os.listdir(folder_path):
            if not file.endswith(".wav"):
                continue
            parts = file.split("-")
            if len(parts) < 3:
                continue
            emotion_code = parts[2]
            label = ravdess_emotion_map.get(emotion_code)
            if not label:
                continue
            file_path = os.path.join(folder_path, file)
            y = load_and_preprocess(file_path)
            features = extract_features(y, SAMPLE_RATE)
            sequences.append(features)
            labels.append(label)
            count += 1
    print(f"✔️ {tag} - Loaded {count} samples")
    return sequences, labels



ravdess_speech_path='/Users/yathamlohithreddy/Desktop/vscodefloder /marsproject/Audio_Speech_Actors_01-24'
ravdess_song_path='/Users/yathamlohithreddy/Desktop/vscodefloder /marsproject/Audio_Song_Actors_01-24'
# Process speech and song separately
speech_sequences, speech_labels = process_ravdess(ravdess_speech_path, tag="Speech")
song_sequences, song_labels = process_ravdess(ravdess_song_path, tag="Song")

# Define emotion code mapping




✔️ Speech - Loaded 1440 samples
✔️ Song - Loaded 1012 samples


In [18]:
max_len = 300  # or any number based on your preferred sequence length

#  Pad separately
X_speech = pad_sequences(speech_sequences, maxlen=max_len, dtype='float32', padding='post', truncating='post')
X_song = pad_sequences(song_sequences, maxlen=max_len, dtype='float32', padding='post', truncating='post')

# Encode separately
le_speech = LabelEncoder()
y_speech = le_speech.fit_transform(speech_labels)

le_song = LabelEncoder()
y_song = le_song.fit_transform(song_labels)


In [19]:
def split_and_scale(X, y):
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42)

    # Scale
    scaler = StandardScaler()
    X_train_2d = X_train.reshape(-1, X.shape[2])
    scaler.fit(X_train_2d)

    X_train = scaler.transform(X_train.reshape(-1, X.shape[2])).reshape(X_train.shape)
    X_val   = scaler.transform(X_val.reshape(-1, X.shape[2])).reshape(X_val.shape)
    X_test  = scaler.transform(X_test.reshape(-1, X.shape[2])).reshape(X_test.shape)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

# Speech split
X_train_sp, X_val_sp, X_test_sp, y_train_sp, y_val_sp, y_test_sp = split_and_scale(X_speech, y_speech)

# Song split
X_train_sg, X_val_sg, X_test_sg, y_train_sg, y_val_sg, y_test_sg = split_and_scale(X_song, y_song)


In [33]:

from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, SpatialDropout1D, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, SpatialDropout1D, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import regularizers


def build_model(timesteps, feat_dim, num_emotions):
    model = Sequential([
        Input(shape=(timesteps, feat_dim)),

        Conv1D(128, kernel_size=9, activation='relu', padding='same',
               kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        SpatialDropout1D(0.3),

        Conv1D(128, kernel_size=7, activation='relu', padding='same',
               kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        SpatialDropout1D(0.3),

        Conv1D(128, kernel_size=5, activation='relu', padding='same',
               kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        SpatialDropout1D(0.3),

        Conv1D(128, kernel_size=3, activation='relu', padding='same',
               kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        SpatialDropout1D(0.3),

        Conv1D(64, kernel_size=1, activation='relu', padding='same',
               kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        SpatialDropout1D(0.3),

        GlobalAveragePooling1D(),

        Dense(num_emotions, activation='softmax')
    ])

    # Optimizer with exponential decay
    lr_schedule = ExponentialDecay(
        initial_learning_rate=1e-4,
        decay_steps=1000,
        decay_rate=0.9
    )

    model.compile(
        optimizer=Adam(learning_rate=lr_schedule),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model




In [34]:
model_speech = build_model(X_train_sp.shape[1], X_train_sp.shape[2], len(le_speech.classes_))
model_song = build_model(X_train_sg.shape[1], X_train_sg.shape[2], len(le_song.classes_))


model_song.summary()
model_speech.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_42 (Conv1D)              │ (None, 300, 128)       │        19,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_42            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_43 (Conv1D)              │ (None, 300, 128)       │       114,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_43            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_44 (Conv1D)              │ (None, 300, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_44            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_45 (Conv1D)              │ (None, 300, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_45            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_46 (Conv1D)              │ (None, 300, 64)        │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 300, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_46            │ (None, 300, 64)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_10     │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 276,806 (1.06 MB)

 Trainable params: 275,654 (1.05 MB)

 Non-trainable params: 1,152 (4.50 KB)

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_37 (Conv1D)              │ (None, 300, 128)       │        19,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_37            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_38 (Conv1D)              │ (None, 300, 128)       │       114,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_38            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_39 (Conv1D)              │ (None, 300, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_39          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_39            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_40 (Conv1D)              │ (None, 300, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_40            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_41 (Conv1D)              │ (None, 300, 64)        │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 300, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_41            │ (None, 300, 64)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_9      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 276,936 (1.06 MB)

 Trainable params: 275,784 (1.05 MB)

 Non-trainable params: 1,152 (4.50 KB)

In [35]:


# Callbacks
cb = [ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True)]

# Train speech model
model_speech.fit(X_train_sp, y_train_sp, validation_data=(X_val_sp, y_val_sp), epochs=100, batch_size=32, callbacks=cb)

# Train song model
model_song.fit(X_train_sg, y_train_sg, validation_data=(X_val_sg, y_val_sg), epochs=100, batch_size=32, callbacks=cb)


Epoch 1/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.1179 - loss: 2.3276

27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - accuracy: 0.1182 - loss: 2.3267 - val_accuracy: 0.1354 - val_loss: 2.1395
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.1083 - loss: 2.2749 - val_accuracy: 0.1354 - val_loss: 2.1685
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.1368 - loss: 2.1839 - val_accuracy: 0.1354 - val_loss: 2.1875
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.1518 - loss: 2.2320 - val_accuracy: 0.1354 - val_loss: 2.2032
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.1621 - loss: 2.2006 - val_accuracy: 0.1458 - val_loss: 2.2117
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.1709 - loss: 2.1466 - val_accuracy: 0.1875 - val_loss: 2.2023
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.1656 - loss: 2.1474 - val_accuracy: 0.1875 - val_loss: 2.1857
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.1971 - loss: 2.1079 - val_accuracy: 0.2500 - val_l

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.1984 - loss: 2.0576 - val_accuracy: 0.2049 - val_loss: 2.1272
Epoch 11/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.2013 - loss: 2.0309

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.2017 - loss: 2.0303 - val_accuracy: 0.2431 - val_loss: 2.1126
Epoch 12/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.1908 - loss: 2.0337

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.1914 - loss: 2.0334 - val_accuracy: 0.2431 - val_loss: 2.0884
Epoch 13/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.2367 - loss: 2.0072

...

19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - accuracy: 0.6095 - loss: 1.1178 - val_accuracy: 0.5714 - val_loss: 1.1145
Epoch 95/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.5967 - loss: 1.1418

19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - accuracy: 0.5982 - loss: 1.1398 - val_accuracy: 0.5862 - val_loss: 1.1006
Epoch 96/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.5940 - loss: 1.1207

19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - accuracy: 0.5951 - loss: 1.1197 - val_accuracy: 0.5911 - val_loss: 1.0964
Epoch 97/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.6531 - loss: 1.0689

19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - accuracy: 0.6518 - loss: 1.0695 - val_accuracy: 0.5813 - val_loss: 1.0917
Epoch 98/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - accuracy: 0.6759 - loss: 1.0414 - val_accuracy: 0.5714 - val_loss: 1.0939
Epoch 99/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - accuracy: 0.6290 - loss: 1.0646 - val_accuracy: 0.5961 - val_loss: 1.0950
Epoch 100/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.6388 - loss: 1.0564

19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - accuracy: 0.6390 - loss: 1.0569 - val_accuracy: 0.5961 - val_loss: 1.0698


In [43]:
def build_model2(timesteps, feat_dim, num_classes):
    model = Sequential([
        Input(shape=(timesteps, feat_dim)),

        Conv1D(128, kernel_size=9, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        SpatialDropout1D(0.3),

        Conv1D(128, kernel_size=7, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        SpatialDropout1D(0.3),

        Conv1D(128, kernel_size=5, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        SpatialDropout1D(0.3),

        Conv1D(64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        SpatialDropout1D(0.3),

        GlobalAveragePooling1D(),
        Dense(num_classes, activation='softmax')
    ])
    
    schedule = ExponentialDecay(1e-4, decay_steps=1000, decay_rate=0.9)
    model.compile(optimizer=Adam(schedule), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [44]:
model_speech2 = build_model2(X_train_sp.shape[1], X_train_sp.shape[2], len(le_speech.classes_))
model_song2 = build_model2(X_train_sg.shape[1], X_train_sg.shape[2], len(le_song.classes_))


model_song.summary()
model_speech.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_51 (Conv1D)              │ (None, 300, 128)       │        19,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_51          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_51            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_52 (Conv1D)              │ (None, 300, 128)       │       114,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_52          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_52            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_53 (Conv1D)              │ (None, 300, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_53          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_53            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_54 (Conv1D)              │ (None, 300, 64)        │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_54          │ (None, 300, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_54            │ (None, 300, 64)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_12     │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 728,403 (2.78 MB)

 Trainable params: 242,502 (947.27 KB)

 Non-trainable params: 896 (3.50 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_47 (Conv1D)              │ (None, 300, 128)       │        19,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_47            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_48 (Conv1D)              │ (None, 300, 128)       │       114,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_48          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_48            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_49 (Conv1D)              │ (None, 300, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_49          │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_49            │ (None, 300, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_50 (Conv1D)              │ (None, 300, 64)        │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_50          │ (None, 300, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_50            │ (None, 300, 64)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_11     │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 728,793 (2.78 MB)

 Trainable params: 242,632 (947.78 KB)

 Non-trainable params: 896 (3.50 KB)

 Optimizer params: 485,265 (1.85 MB)

In [47]:


# Callbacks
cb = [ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True)]

# Train speech model
model_speech2.fit(X_train_sp, y_train_sp, validation_data=(X_val_sp, y_val_sp), epochs=100, batch_size=32, callbacks=cb)

# Train song model
model_song2.fit(X_train_sg, y_train_sg, validation_data=(X_val_sg, y_val_sg), epochs=100, batch_size=32, callbacks=cb)


Epoch 1/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6242 - loss: 1.2401

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.6248 - loss: 1.2391 - val_accuracy: 0.5486 - val_loss: 1.3461
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.6010 - loss: 1.2478

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.6017 - loss: 1.2469 - val_accuracy: 0.5694 - val_loss: 1.3173
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.6177 - loss: 1.2154 - val_accuracy: 0.5556 - val_loss: 1.3292
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.6157 - loss: 1.1968

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.6159 - loss: 1.1969 - val_accuracy: 0.5868 - val_loss: 1.2924
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.6347 - loss: 1.2037

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.6355 - loss: 1.2033 - val_accuracy: 0.5938 - val_loss: 1.2765
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.6100 - loss: 1.2062 - val_accuracy: 0.5799 - val_loss: 1.2822
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6658 - loss: 1.1625

...

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.8162 - loss: 0.7439 - val_accuracy: 0.7292 - val_loss: 0.9059
Epoch 81/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.8261 - loss: 0.7077 - val_accuracy: 0.7014 - val_loss: 0.9220
Epoch 82/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.7734 - loss: 0.7528 - val_accuracy: 0.6944 - val_loss: 0.9425
Epoch 83/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.8106 - loss: 0.7338 - val_accuracy: 0.6944 - val_loss: 0.9835
Epoch 84/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.7760 - loss: 0.7949 - val_accuracy: 0.7118 - val_loss: 0.9159
Epoch 85/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.8302 - loss: 0.7056

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.8301 - loss: 0.7058 - val_accuracy: 0.7014 - val_loss: 0.9032
Epoch 86/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.8162 - loss: 0.7099 - val_accuracy: 0.7257 - val_loss: 0.9129
Epoch 87/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.8414 - loss: 0.7036 - val_accuracy: 0.7118 - val_loss: 0.9099
Epoch 88/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.8193 - loss: 0.7128 - val_accuracy: 0.7083 - val_loss: 0.9411
Epoch 89/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.8198 - loss: 0.7209 - val_accuracy: 0.7222 - val_loss: 0.9060
Epoch 90/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.8069 - loss: 0.7174 - val_accuracy: 0.6979 - val_loss: 0.9314
Epoch 91/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.8018 - loss: 0.7075

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.8023 - loss: 0.7066 - val_accuracy: 0.7188 - val_loss: 0.9014
Epoch 92/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.8222 - loss: 0.6757 - val_accuracy: 0.7049 - val_loss: 0.9040
Epoch 93/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.8320 - loss: 0.6628 - val_accuracy: 0.7326 - val_loss: 0.9086
Epoch 94/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.8578 - loss: 0.6439

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.8569 - loss: 0.6449 - val_accuracy: 0.7222 - val_loss: 0.8947
Epoch 95/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.8350 - loss: 0.6741 - val_accuracy: 0.7083 - val_loss: 0.9089
Epoch 96/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.8152 - loss: 0.6793 - val_accuracy: 0.7049 - val_loss: 0.9231
Epoch 97/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.8055 - loss: 0.7052 - val_accuracy: 0.6944 - val_loss: 0.9190
Epoch 98/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.8406 - loss: 0.6964 - val_accuracy: 0.7083 - val_loss: 0.9023
Epoch 99/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.8413 - loss: 0.6436

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.8412 - loss: 0.6437 - val_accuracy: 0.6979 - val_loss: 0.8808
Epoch 100/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.8515 - loss: 0.6689 - val_accuracy: 0.7049 - val_loss: 0.8842
Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.1747 - loss: 1.9414 - val_accuracy: 0.1823 - val_loss: 1.8230
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.1396 - loss: 1.9144 - val_accuracy: 0.1823 - val_loss: 1.8156
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.1974 - loss: 1.8722 - val_accuracy: 0.1921 - val_loss: 1.8047
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.2051 - loss: 1.8186 - val_accuracy: 0.2167 - val_loss: 1.7986
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.2614 - loss: 1.7533 - val_accuracy: 0.2266 - val_loss: 1.7868
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.2885 - loss: 1.7166 - val_accuracy: 0.2414 - val

In [48]:
import matplotlib.pyplot as plt

def compare_two_models(hist1, hist2, label1, label2, title_prefix="Model"):
    epochs = range(1, len(hist1.history['loss']) + 1)

    plt.figure(figsize=(14, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs, hist1.history['val_accuracy'], label=f'{label1}')
    plt.plot(epochs, hist2.history['val_accuracy'], label=f'{label2}')
    plt.title(f'{title_prefix}: Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, hist1.history['val_loss'], label=f'{label1}')
    plt.plot(epochs, hist2.history['val_loss'], label=f'{label2}')
    plt.title(f'{title_prefix}: Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()


In [49]:
compare_two_models(
    model_speech,model_speech2,
    label1='Speech CNN v1',
    label2='Speech CNN v2',
    title_prefix="Speech Model Comparison"
)


TypeError: 'History' object is not subscriptable

In [ ]:
compare_two_models(
    model_song, model_song2,
    label1='Song CNN v1',
    label2='Song CNN v2',
    title_prefix="Song Model Comparison"
)


TypeError: 'History' object is not subscriptable

In [26]:
# Speech
pred_sp = np.argmax(model_speech.predict(X_test_sp), axis=1)
print("Speech Accuracy:", accuracy_score(y_test_sp, pred_sp))
print(classification_report(y_test_sp, pred_sp, target_names=le_speech.classes_))

# Song
pred_sg = np.argmax(model_song.predict(X_test_sg), axis=1)
print("Song Accuracy:", accuracy_score(y_test_sg, pred_sg))
print(classification_report(y_test_sg, pred_sg, target_names=le_song.classes_))


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


NameError: name 'accuracy_score' is not defined